In [1]:
import numpy as np
import math

In [2]:
S = {
    "B": np.array([1,0,0,0,0]).T,
    "C": np.array([0,1,0,0,0]).T,
    "D": np.array([0,0,1,0,0]).T,
    "E": np.array([0,0,0,1,0]).T,
    "F": np.array([0,0,0,0,1]).T,
}

In [3]:
def p(xt, w, debug=False):
    pt = np.dot(w.T, xt)
    if debug: print("\t\t\t\tpt={0}".format(pt))
    return pt

In [4]:
t = 0
xt = np.array([0,0,0,0,1], dtype=np.float)
w = np.array([0,0,0,0,1], dtype=np.float)
p(xt, w)

1.0

In [5]:
def residual(xt, l, t, k, debug=False):
    #return (l ** (t-k)) * xt
    #is it always 1?
    if debug: print("\t\t\t(t-k)={0}".format((t-k)))
    return (l ** (t-k)) * 1.0

In [6]:
residual(5, l=1.0, t=2, k=1)

1.0

In [7]:
def residual_sum(X, l, debug=False):
    total = 0
    t = len(X)
    for k in range(1, t+1):
        xk = S[X[k-1]]
        res = residual(xk, l, t, k)
        if debug: print("\t\tres={0},total={1},xk={2},lambda={3},xt={4}".format(res,total,xk,l,X[k-1]))
        total += res
    return total

In [8]:
#X = ['D','C','D','E','F','G']
X = ['D','C','D','E','F']

In [9]:
residual_sum(X, l=.5)

1.9375

In [10]:
def error(X,w,t,a, debug=False):
    xt = S[X[t-1]]
    xt_prime = S[X[t]]
    pred = p(xt, w)
    pred_prime = p(xt_prime, w)
    err = a * (pred_prime - pred)
    if debug: print("\txt={0},xt_prime={1}, pred={2}, pred_prime={3}, err={4}".format(X[t-1],X[t],pred,pred_prime,err))
    return err

In [11]:
error(X,w,t=1,a=1.0)

0.0

In [12]:
def delta_w_t(X,w,t,a,l,debug=False):
    err = error(X,w,t,a)
    res = residual_sum(X, l=l)
    dw = err * res
    if debug: ("t={0},err={1},res={2},w={3},dw={4}".format(t,err,res,w,dw))
    return dw

In [13]:
delta_w_t(X,w,t,a=1.0,l=1.0)

-5.0

In [59]:
# accumulated over sequences and 
# only used to update the weight vector 
# after the complete presentation of a training set.
def delta_w_t_v1(Xs,w,a,l, debug=False):
    # Xs == training set (10 sequences)
    # seq is X (sequence)
    Ws = []
    for i in range(len(Xs)):
        X_Z = Xs[i]
        X,_ = X_Z
        for t in range(1, len(X)):
            temp_dw = delta_w_t(X,w,t,a=a,l=l)
            Ws.append(temp_dw)
            if debug: print("t={0}, dw={1}".format(t,temp_dw))
    
    # return new w
    new_w = np.array(Ws)
    new_w = np.sum(new_w, axis=0)
    
    return new_w
    

In [60]:
delta_w_t_v1([(X,0)],w,a=1.0,l=1.0,debug=True)

t=1, dw=0.0
t=2, dw=0.0
t=3, dw=0.0
t=4, dw=5.0


5.0

In [61]:
def gen_sequence():
    T = {
        "B": np.array(['A',"C"]),
        "C": np.array(["B","D"]),
        "D": np.array(["C","E"]),
        "E": np.array(["D","F"]),
        "F": np.array(["E","G"]),
    }
    
    S = "D"
    
    seqs = [S]
    
    while True:
        if S == "A":
            return (seqs,0)
        elif S == 'G':
            return (seqs,1)
        
        #do a random roll
        ns = T[S]
        S = np.random.choice(ns,p=[.5,.5])
        seqs.append(S)
    

In [62]:
gen_sequence()

(['D', 'E', 'F', 'E', 'F', 'G'], 1)

In [65]:
def run_v1(a,l, debug=False):
    #gen 100 training sets
        #each should have 10 sequences
    training_sets = []
    for ts in range(0,100):
        seq = []
        for ss in range(0,10):
            seq.append(gen_sequence())
        training_sets.append(seq)
    
    # now run
    w = [0,0,0,0,0]
    for i,train_set in enumerate(training_sets):
        new_w = delta_w_t_v1(train_set,w,a,l, debug=False)
        if debug: print("train_set_i={0}, old_w={1}, new_w={2}".format(i,w,new_w))
        w = new_w
    
    return w
    

In [66]:
run_v1(a=.12,l=0.0, debug=False)

AttributeError: 'list' object has no attribute 'T'